### Iterative policy evaluation

Implement iterative policy evaluation and use it to estimate 𝑣𝜋
for
the grid world in Exercise gw/simple, where 𝜋 is the equiprobable
random policy.


In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def softmax(x, axis=0):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=axis)

In [19]:
class Grid:
    
    def __init__(self, w, h):
        self._w = w
        self._h = h
        
        self.R = -np.ones((w, h))
        self.actions = {
            "up": 0,
            "down": 1,
            "left": 2,
            "right": 3
        }
        
        self.terminal = [(0,0), (w-1, h-1)]
                    
    def sprime(self, s:np.array, a:int) -> tuple:
        # actions: (0,1,2,3) = (up, down, left, right)
        actions = {
            0: np.array((-1, 0)),
            1: np.array((1, 0)),
            2: np.array((0, -1)),
            3: np.array((0, 1))
        }
        s_prime = s + actions[a]
        s_prime[0] = max(min(s_prime[0], self._w-1), 0)
        s_prime[1] = max(min(s_prime[1], self._h-1), 0)
        
        return tuple(s_prime)

    def reward(self, s:np.array, a: int) -> float:
        s_prime = self.sprime(s, a)
        reward = self.R[s_prime[0], s_prime[1]]

        return reward

    def cumulative_reward(self, s, π, γ=0.9, max_steps=1):
        if s in self.terminal:
            return 0
        reward = 0
        
        for a in range(π.shape[0]):
            sprime = self.sprime(s, a)
            
            if max_steps > 1:
                reward += π[a, s[0], s[1]]*(self.reward(s, a) + γ*self.cumulative_reward(sprime, π, γ, max_steps-1))
            else:
                reward += π[a, s[0], s[1]]*self.reward(s, a)
        return reward

    def iterative_policy_evaluation(self, π, γ=1.0, θ=0.1, max_steps=1000):
        V = np.random.rand(self._w, self._h)      
        for t in self.terminal:
            V[t] = 0

        Δ = θ+1
        steps = 0
        while Δ > θ:
            Δ = 0
            for x in range(self._w):
                for y in range(self._h):
                    s = (x, y)
                    if s in self.terminal:
                        continue
                    
                    v = V[s]
                    v_new = 0
                    
                    # The additional sum over s', r can be ignored in our case because our enviroment is deterministic
                    # i.e. for a given state s and action a the outcome is s'(a) and r(s')
                    for a in range(π.shape[0]):
                        sprime = self.sprime(s, a)
                        v_new += π[a, x, y]*(self.reward(s, a) + γ*V[sprime])
                        
                    V[s] = v_new
                    Δ = max(Δ, abs(v - V[s]))
                    
            steps += 1
            if steps > max_steps:
                print(f'Reached max step. {Δ=}.')
                return V
            
        print(f'{steps=}')
        return V

In [20]:
w, h = 4, 4

g = Grid(w, h)

In [21]:
# A policy function is a matrix "policy" with shape (#actions, grid_w, grid_h) 
# where policy[a, x, y] is the probability of adction a in state (x, y).
# actions: (1,2,3,4) = (up, down, left, right)

# π = softmax(np.random.rand(4, w, h)); # random policy
π = softmax(np.ones((4, w, h))); # equiprob policy

In [22]:
g.iterative_policy_evaluation(π, γ=0, θ=0.001)
# plt.imshow(g.iterative_policy_evaluation(π, γ=0, θ=0.001))

steps=2


array([[ 0., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1.,  0.]])

In [23]:
g.iterative_policy_evaluation(π, γ=1, θ=0.001, max_steps=200)
# plt.imshow(g.iterative_policy_evaluation(π, γ=0.99, θ=0.001))

steps=88


array([[  0.        , -13.99314976, -19.99015187, -21.9891603 ],
       [-13.99314976, -17.99159386, -19.99087286, -19.99097723],
       [-19.99015187, -19.99087286, -17.99229837, -13.99424987],
       [-21.98916031, -19.99097723, -13.99424987,   0.        ]])

In [8]:
#optimal policy
π = softmax(np.ones((4, w, h))); # equiprob policy
for x in range(w):
    for y in range(h):
        dy = (h-1) - y
        dx = (w-1) - x
        
        if min(x+y,dx+dy) == x+y:
            if y==0:
                π[:,x,y] = (1,0,0,0)
            else:
                π[:,x,y] = (0,0,1,0)
                
        elif min(x+y,dx+dy) == dx+dy:
            if dy==0:
                π[:,x,y] = (0,1,0,0)
            else:
                π[:,x,y] = (0,0,0,1)

In [9]:
g.iterative_policy_evaluation(π, γ=0, θ=0.001)
# plt.imshow(g.iterative_policy_evaluation(π, γ=0, θ=0.001))

steps=2


array([[ 0., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1., -1.],
       [-1., -1., -1.,  0.]])

In [10]:
g.iterative_policy_evaluation(π, γ=1, θ=0.001)
# plt.imshow(g.iterative_policy_evaluation(π, γ=1, θ=0.001))

steps=3


array([[ 0., -1., -2., -3.],
       [-1., -2., -3., -2.],
       [-2., -3., -2., -1.],
       [-3., -2., -1.,  0.]])